# L3: Agentic Sales Pipeline

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
# from helper import load_env
# load_env()

import os
import yaml
from crewai import Agent, Task, Crew,LLM

import nest_asyncio
nest_asyncio.apply()

os.environ['SERPER_API_KEY'] ="30bdec95f601d48ca27880feedef448ca840b2f0"

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Load API tokens for our 3rd party APIs

In [3]:
# os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"


llm = LLM(model="ibm-granite/granite-3.3-2b-instruct", 
          base_url=f"{INFERENCE_SERVER_URL}/v1",
          api_key=API_KEY,
          custom_llm_provider ="openai",
          max_tokens=1024)

In [4]:
config = dict(
    llm=dict(
        provider="openai",
        config=dict(
            model="ibm-granite/granite-3.3-2b-instruct",
            base_url=f"{INFERENCE_SERVER_URL}/v1",
            api_key=API_KEY,
        ),
    ),
    # embedder=dict(
    #     provider="huggingface",#,.goolge
    #     config=dict(
    #         model="BAAI/bge-small-en-v1.5"#"models/embedding-001"#"nomic-ai/nomic-embed-text-v1"
    #     ),
    # ),
)

## Loading Tasks and Agents YAML files

In [5]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'L3-config/lead_qualification_agents.yaml',
    'lead_tasks': 'L3-config/lead_qualification_tasks.yaml',
    'email_agents': 'L3-config/email_engagement_agents.yaml',
    'email_tasks': 'L3-config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic Models for Structured Output

In [6]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Importing Tools

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

/opt/app-root/lib64/python3.11/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## Lead Qualification Crew, Agents and Tasks

In [8]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  llm=llm,
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  llm=llm,
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  llm=llm,
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [9]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist'],
  llm=llm
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist'],
  llm=llm
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

## Creating Complete Sales Flow

In [10]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: SalesPipeline                                                                                            │
│  ID: 906e84dc-af4b-4ffa-b532-415cf7afab9a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Plotting the Flow

In [11]:
flow.plot()

Plot saved as crewai_flow.html


In [12]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [13]:
# emails = await flow.kickoff()
emails =  flow.kickoff()

 Flow started with ID: 906e84dc-af4b-4ffa-b532-415cf7afab9a

Output()

Provider List: https://docs.litellm.ai/docs/providers

Provider List: https://docs.litellm.ai/docs/providers

Provider List: https://docs.litellm.ai/docs/providers

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Engagement Optimization Specialist                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Transform Your Enterprise with CrewAI's Multi-Agent Orchestration                                     │
│                                                                                                                 │
│  Hi there,                                                                                                      │
│                                                                                                                 │
│  Thanks for filling out your lead form with CrewAI – we're excited to help enterprise companies like yours      │
│  revolutionize automation with our platform.                                                                    │
│                                                                                                                 │
│  Imagine having a single system to orchestrate AI agents across any vertical. That's exactly what CrewAI        │
│  offers. Our innovative Multi-Agent Orchestration Platform is designed to streamline complex automations and    │
│  let you focus on your core business objectives.                                                                │
│                                                                                                                 │
│  Now, are you ready to see how CrewAI can transform your operations? Schedule a quick meeting with our team to  │
│  explore tailored solutions for your organization.                                                              │
│                                                                                                                 │
│  Click here to book your meeting or reply to this email for immediate assistance.                               │
│                                                                                                                 │
│  Best,                                                                                                          │
│  CrewAI Team                                                                                                    │
│                                                                                                                 │
│  P.S. Don't miss out on this chance to optimize your workflows with cutting-edge AI automation – take action    │
│  now!                                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b7e90e5f-062d-4c9c-bae4-c3078bb8639f                                                                     │
│  Agent: Engagement Optimization Specialist                                                                      │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 62624698-b9fe-4137-a337-82d1ecb0c940                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Transform Your Enterprise with CrewAI's Multi-Agent Orchestration                       │
│                                                                                                                 │
│  Hi there,                                                                                                      │
│                                                                                                                 │
│  Thanks for filling out your lead form with CrewAI – we're excited to help enterprise companies like yours      │
│  revolutionize automation with our platform.                                                                    │
│                                                                                                                 │
│  Imagine having a single system to orchestrate AI agents across any vertical. That's exactly what CrewAI        │
│  offers. Our innovative Multi-Agent Orchestration Platform is designed to streamline complex automations and    │
│  let you focus on your core business objectives.                                                                │
│                                                                                                                 │
│  Now, are you ready to see how CrewAI can transform your operations? Schedule a quick meeting with our team to  │
│  explore tailored solutions for your organization.                                                              │
│                                                                                                                 │
│  Click here to book your meeting or reply to this email for immediate assistance.                               │
│                                                                                                                 │
│  Best,                                                                                                          │
│  CrewAI Team                                                                                                    │
│                                                                                                                 │
│  P.S. Don't miss out on this chance to optimize your workflows with cutting-edge AI automation – take action    │
│  now!                                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: SalesPipeline                                                                                            │
│  ID: 906e84dc-af4b-4ffa-b532-415cf7afab9a                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [14]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0023


/tmp/ipykernel_2713/1963678899.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,15053,13080,0,1973,8


In [15]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
# df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# # Calculate total costs
# costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
# print(f"Total costs: ${costs:.4f}")

# # Display the DataFrame
# df_usage_metrics

## Inspecting Results

In [16]:
scores = flow.state["score_crews_results"]

In [17]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,João Moura
Job Title,Director of Engineering
Role Relevance,8
Professional Background,"Likely responsible for leading engineering teams focused on data enrichment solutions, aligning with CrewAI's AI agent-driven use cases."
Company Name,Clearbit
Industry,Business Software and Services
Company Size,100000
Revenue,100000000.000000
Market Presence,9
Lead Score,75


## Results

In [18]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

Subject: Transform Your Enterprise with CrewAI's Multi-Agent Orchestration  Hi
there,  Thanks for filling out your lead form with CrewAI – we're excited to
help enterprise companies like yours revolutionize automation with our platform.
Imagine having a single system to orchestrate AI agents across any vertical.
That's exactly what CrewAI offers. Our innovative Multi-Agent Orchestration
Platform is designed to streamline complex automations and let you focus on your
core business objectives.  Now, are you ready to see how CrewAI can transform
your operations? Schedule a quick meeting with our team to explore tailored
solutions for your organization.   Click here to book your meeting or reply to
this email for immediate assistance.  Best, CrewAI Team  P.S. Don't miss out on
this chance to optimize your workflows with cutting-edge AI automation – take
action now!


## How Complex Can it Get?

In [19]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

TypeError: router() got an unexpected keyword argument 'paths'

## Plotting the Flow

In [ ]:
flow = SalesPipeline()
flow.plot()

In [ ]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)